# CAO Points Analysis
### Conor McCaffrey

***



Give info about task

***

Give info about packages you are importing and why, with references

In [1]:
# Convenient HTTP requests.
import requests as rq

# Regular expressions.
import re

# Dates and times.
import datetime as dt

# Data frames. (dataframes is another way to say spreadsheets)
import pandas as pd

# For downloading.
import urllib.request as urlrq

import seaborn as sns

<br>

## 2021 Points

#### Please click below for 2021 courses
http://www.cao.ie/index.php?page=points&p=2021 [1]



Give overview here of response function (concise)

##### Steps to re-produce loading of Data in Pandas Dataframe

1. Use Requests library [] to make HTTP request to Python.
2. Using DateTime package to get current date and time [], create a filepath for data.
3. Rectify encoding error on dataset by setting to 'cp1252'.
4. Re-save Word Document for editing.
5. Save this rectifed HTML dataset.
6. Using regular expressions [], select the information we require from HTML file.
7. Loop through this file, using regular expression and save as a .csv file.
8. Read this .csv file into a Pandas dataframe
9. Call a method on the dataframe to ensure it is loaded correctly



##### Let's gather the Level 8 courses

In [2]:
# Fetch the CAO Level 8 points URL.  (based on Lecture Notes)
resp8 = rq.get('http://www2.cao.ie/points/l8.php')
# Have a quick look to ensure all is okay
resp8       # Response 200 means all is okay. 404 means not found

<Response [200]>

##### Let's gather the Level 7/6 courses

In [3]:
# Fetch the CAO Level 7/6 points URL (adapted from Lecture Notes)
resp7 = rq.get('http://www2.cao.ie/points/l76.php')
# Have a quick look to ensure all is okay
resp7      # Response 200 means all is okay. 404 means not found

<Response [200]>

<br>

## Save original dataset

In [4]:
# Get current date and time using Datetime Package (https://docs.python.org/3/library/datetime.html)
now = dt.datetime.now()

# Format as a string.
nowstr = now.strftime('%Y%m%d_%H%M%S')

In [5]:
# Create a filepath for original Level 8 data (https://automatetheboringstuff.com/chapter8/)
pathhtml8 = 'data/cao2021_Lvl8_' +  nowstr + '.html'
# Create a filepath for original Level 7/6 data
pathhtml7 = 'data/cao2021_Lvl76_' +  nowstr + '.html'

<br>

## So we have an error on server

***

Technically the server says we should decode as iso-8859

```
Content-Type: text/html; charset=iso-8859-1

```

However, one line uses \x96 which isnt defined in iso-8859-1. <br>
Therefore, we use the similar decoding standard cp1252 which is similar but also includes \x96.

In [6]:
## Level 8
# The server uses the wrong encoding, fix it
original_encoding = resp8.encoding
# Change it to cp1252
resp8.encoding = 'cp1252'

In [7]:
# Save the original HTML file
with open(pathhtml8, 'w') as f:
    f.write(resp8.text)

In [8]:
## Level 7/6
# The server uses the wrong encoding, fix it
original_encoding = resp7.encoding
# Change it to cp1252
resp7.encoding = 'cp1252'

In [9]:
# Save the original HTML file
with open(pathhtml7, 'w') as f:
    f.write(resp7.text)

# Use regular expressions to select lines we want

***

In [10]:
# Compile the regular expression for matching lines. (https://docs.python.org/3/library/re.html)
re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)') 


### Loop through the lines of the response

***

In [11]:
# Write a helper function

#def points_to_array(s):  # take in the strings with points numbers
 #   portfolio = ''
 #   if s[0] == '#':
  #      portfolio = '#'
   # random = ''
    #if s[-1] == '*':
    #    random = '*'
    #points = ''
    #for i in s:
     #   if i.isdigit():
      #      points = points + i
    #return [points, portfolio, random]
        

In [12]:
#The filepath for the csv file. Level 8

path2021_8 = 'data/cao2021_Lvl8_' +  nowstr + '.csv'



In [13]:
# keep track of how many courses we process  COMMENT THIS
no_lines = 0

# Open the csv file for writing.
try:
    with open (path2021_8, 'w') as f:
        # Write a header row.
        f.write(','.join(['Code', 'Title', 'PointsR1', 'PointsR2']) + '\n')
        # Loop through the lines of the response.
        for line in resp8.iter_lines(): # took from google stack overflow.
        # Decode line, using the wrong encoding!
            dline = line.decode('cp1252')
        # match only the lines we want - the ones representing courses.
            if re_course.fullmatch(dline):    # getting set up for regular expression. 'pattern is the regular expression itself'
            # Add one to the lines counter
                no_lines = no_lines + 1
            # Pick out the relevant parts of the matched line
             #   csv_version = re_course.sub(r'\1,\2,\3,\4', dline) # go in and one reg expres, what we want back isgroups of regular expression (/1 is first part, /2 is second part and so on. Reg exp above has 4 parts.)  
            # split the line on two or more spaces
              #  linesplit = re.split('  +', dline) # twos space and plus means two or more space
            # Debugging
               # print(len(linesplit), linesplit, dline)
            # The course code
                course_code = dline[:5]
            # The course title
                course_title = dline[7:57].strip()
            # Round 1 points
                course_points = re.split(' +', dline[60:])
                if len(course_points) != 2:
                    course_points = course_points[:2]
            # Join the fields using a comma.
                linesplit = [course_code, course_title, course_points[0], course_points[1]] # make last two things into arrays with 3 things.
            # rejoin the substrings with commas in between.
                f.write(','.join(linesplit) + '\n')
except BaseException as err:
    print('This has caused an error')
else:
    print('Process has been successful.',f'Total number of lines is {no_lines}.')          




Process has been successful. Total number of lines is 949.


<br>

<b> ** N.B ** </b> It was verified as of 08/11/2021 there were 949 courses exactly in the CAO 2021 Points List.

***

In [14]:
df2021_8 = pd.read_csv(path2021_8, encoding='cp1252') 

In [15]:
df2021_8

,Code,Title,PointsR1,PointsR2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


In [16]:
# Reading in csv file and converting to pandas dataframe (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)
#df2021 = pd.read_csv('data/cao2021_Lvl8_20211101_201121.csv', sep=',') # consider making a table listing University and the letters for it

In [17]:
#The filepath for the csv file.

path2021_7 = 'data/cao2021_Lvl76_' +  nowstr + '.csv'
# keep track of how many courses we process
no_lines = 0

# Open the csv file for writing.
try:
    with open (path2021_7, 'w') as f:
        # Write a header row.
        f.write(','.join(['Code', 'Title', 'PointsR1', 'PointsR2']) + '\n')
        # Loop through the lines of the response.
        for line in resp7.iter_lines(): # took from google stack overflow.
            # Decode line, using the wrong encoding!
            dline = line.decode('cp1252')
            # match only the lines we want - the ones representing courses.
            if re_course.fullmatch(dline):    # getting set up for regular expression. 'pattern is the regular expression itself'
                # Add one to the lines counter
                no_lines = no_lines + 1
                #Uncomment next line to see the original
                #print(line)   # the .* is filler, can be basically anything. we are using it for the course name and spaces at end
                # Pick out the relevant parts of the matched line
                #csv_version = re_course.sub(r'\1,\2,\3,\4', dline) # go in and one reg expres, what we want back is groups of regular expression (/1 is first part, /2 is second part and so on. Reg exp above has 4 parts.)  
                # Print the CSV-style line
                #print(csv_version)
                # split the line on two or more spaces
                #linesplit = re.split('  +', dline) # twos space and plus means two or more space
                # Debugging
                    # print(len(linesplit), linesplit, dline)
                # The course code
                course_code = dline[:5]
                # The course title
                course_title = dline[7:57].strip()
                # Round 1 points
                course_points = re.split(' +', dline[60:])
                if len(course_points) != 2:
                    course_points = course_points[:2]
                # Join the fields using a comma.
                linesplit = [course_code, course_title, course_points[0], course_points[1]] # make last two things into arrays with 3 things.
                # rejoin the substrings with commas in between.
                f.write(','.join(linesplit) + '\n')
except BaseException as err:
    print('This has caused an error')
else:
    print('Process has been successful.',f'Total number of lines is {no_lines}.')    


Process has been successful. Total number of lines is 416.


In [18]:
df2021_7 = pd.read_csv(path2021_7, encoding='cp1252') 

In [19]:
# Reading in csv file and converting to pandas daraframe (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)
#df2021_7 = pd.read_csv('data/cao2021_Lvl76_20211101_205701.csv', sep=',') # consider making a table listing University and the letters for it

In [20]:
# lets ensure it calls out correctly
df2021_7

,Code,Title,PointsR1,PointsR2
0,AL605,Music and Instrument Technology,211,NaN
1,AL630,Pharmacy Technician,308,NaN
2,AL631,Dental Nursing,311,NaN
3,AL632,Applied Science,297,NaN
4,AL650,Business,AQA,AQA
...,...,...,...,...
411,WD188,Applied Health Care,220,NaN
412,WD205,Molecular Biology with Biopharmaceutical Science,AQA,262v
413,WD206,Electronic Engineering,180,NaN
414,WD207,Mechanical Engineering,172,NaN


<br>

## 2020 Points

#### Please click below for 2020 courses
http://www.cao.ie/index.php?page=points&p=2020

***





In [21]:
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

##### Steps to re-produce loading of Data in Pandas Dataframe

1. Using DateTime package, create filepath for data in excel format.
2. Save this file to disk using 'path' variable and url requests (https://docs.python.org/3/library/urllib.request.html).
3. Read data into Pandas Dataframe, using skiprows to remove unecessary data.
4. Spot check the data to ensure it has all transferred correctly.
5. Creat filepath for dataframe in excel .csv format.
6. Save Pandas dataframe to disk.


<br>

### Save Original Dataset

***

#### Let's gather the Data

In [22]:
# Create a filepath for original data. 
path2020 = 'data/cao2020_' +  nowstr + '.xlsx'

In [23]:
# Save original file to disk.
urlrq.urlretrieve(url2020, path2020) 

('data/cao2020_20211109_195814.xlsx',
 <http.client.HTTPMessage at 0x2a91af3b880>)

<br>

#### Load Spreadsheet using Pandas

***

In [24]:
# Download and parse the excel spreadsheet
# Read excel function turnsnit into a dataframe in memory so we are now dealing with pandas df from here on
df2020 = pd.read_excel(url2020, skiprows = 10) # pandas dataframe

In [25]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df2020.iloc[753] # in Pandas, iloc gets i-location (row location). loc works for name. Spot check, random row

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Road Transport Technology and Management
COURSE CODE2                                                           LC286
R1 POINTS                                                                264
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      264
EOS Random *                                                             NaN
EOS Mid-point                                                            360
LEVEL                                                                      7
HEI                                         Limerick Institute of Technology
Test/Interview #                                                         NaN

In [27]:
df2020.iloc[1463]  # checking final row. A -1 would work as well (slicing) Spot check, final row

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [28]:
# Create a filepath for the Pandas data
path = 'data/cao2020_' +  nowstr + '.csv'

In [29]:
# Save Pandas dataframe to disk. Look up to_csv to get rid of row index
df2020.to_csv(path)

In [30]:
df2020.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

<br>

Dont forget Level 7/6 points
Camelot or tabula-py (depends on Java)

## 2019 Points

http://www.cao.ie/index.php?page=points&p=2019

***

##### Steps to re-produce 

1. Download original PDF file.
2. Open original PDF file in Microsoft Word.
3. Save Microsoft Word's converted PDF in docx format.
4. Re-save Word Document for editing.
5. Delete Headers and Footers.
6. Delete preamble on Page 1.
7. Select all and copy.
8. Paste into Notepad ++.
9. Remove HEI name headings and paste onto each course line.
10. Delete blank lines.
11. Replace double tab characters with a single tab character (6 occurences)
12. Deleted tab characters at end of lines
13. Change backticks to apostrophes

In [31]:
df2019 = pd.read_csv('data/cao2019_20211108_223304.csv', sep = '\t') # consider making a table listing University and the letters for it
# then merge dataframes that linked course codes to HEIs
df2019

,Course Code,Course,EOS,Mid
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,AL802,Software Design with Cloud Computing,301,306.0
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,AL805,Network Management and Cloud Infrastructure,329,442.0
4,AL810,Quantity Surveying,307,349.0
...,...,...,...,...
925,WD200,Arts (options),221,296.0
926,WD210,Software Systems Development,271,329.0
927,WD211,Creative Computing,275,322.0
928,WD212,Recreation and Sport Management,274,311.0


In [32]:
df2019.iloc[900:926]

,Course Code,Course,EOS,Mid
900,WD048,Business (options),278,357.0
901,WD084,Accounting,307,403.0
902,WD091,Hospitality Management,210,281.0
903,WD116,General Nursing,413,430.0
904,WD117,Psychiatric Nursing,361,382.0
905,WD120,Intellectual Disability Nursing,338,359.0
906,WD134,International Business,289,350.0
907,WD137,Design (Visual Communications),253,373.0
908,WD140,Laws,290,352.0
909,WD144,Architecture,283,385.0


In [33]:
df2019_l7 = pd.read_csv('data/cao2019_20211101_221930_edited.CSV', sep = '\t') # consider making a table listing University and the letters for it
# then merge dataframes that linked course codes to HEIs

In [34]:
df2019_l7.sort_values('Course Code')

,Course Code,Course,EOS,Mid
0,AL600,Software Design,205,306.0
1,AL601,Computer Engineering,196,272.0
2,AL602,Mechanical Engineering,258,424.0
3,AL604,Civil Engineering,252,360.0
4,AL630,Pharmacy Technician,306,366.0
...,...,...,...,...
456,WD188,Applied Health Care,206,339.0
457,WD205,Molecular Biology with Biopharmaceutical Science,208,441.0
458,WD206,Electronic Engineering,191,322.0
459,WD207,Mechanical Engineering,179,330.0


## Analysis of CAO 2021 Level 8,7,6 Points


In [35]:
#Lets first understand the basic information about this Level 8 data
df2021_8.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 949 entries, 0 to 948
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Code      949 non-null    object
 1   Title     949 non-null    object
 2   PointsR1  928 non-null    object
 3   PointsR2  258 non-null    object
dtypes: object(4)
memory usage: 29.8+ KB


In [36]:
#shape can be used to get the shape of dataframe
#columns gives us the list of columns in the dataframe

# Information on the points from 2021 Level 8
df2021_8.describe()


,Code,Title,PointsR1,PointsR2
count,949,949,928,258
unique,949,709,377,180
top,GA480,Business,300,613*
freq,1,12,18,5


In [37]:
df2021_8.iloc[431] 

Code                                       TU992
Title       Community Development and Youth Work
PointsR1                                     251
PointsR2                                     NaN
Name: 431, dtype: object

In [38]:
genre = df2021_8['Code']
genre

0      AL801
1      AL802
2      AL803
3      AL805
4      AL810
       ...  
944    WD211
945    WD212
946    WD230
947    WD231
948    WD232
Name: Code, Length: 949, dtype: object

In [39]:
# Extract data as dataframe
genr = df2021_8[['PointsR1']]
genr
type('genr')

str

In [40]:
# Use regular expression to capture asteriks/hash and replace 
#(https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.Series.str.replace.html) 

#df2021_8['PointsR1'] = df2021_8['PointsR1'].str.replace(r'\*', '', regex=True)
#df2021_8['PointsR1'] = df2021_8['PointsR1'].str.replace(r'\#', '', regex=True)


In [41]:
# Use regular expression to capture asteriks/hash and replace 
#(https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.Series.str.replace.html) 
#df2021_8['PointsR2'] = df2021_8['PointsR2'].str.replace(r'\*', '', regex=True)
#df2021_8['PointsR2'] = df2021_8['PointsR2'].str.replace(r'\#', '', regex=True)

In [42]:
df2021_8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 949 entries, 0 to 948
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Code      949 non-null    object
 1   Title     949 non-null    object
 2   PointsR1  928 non-null    object
 3   PointsR2  258 non-null    object
dtypes: object(4)
memory usage: 29.8+ KB


In [43]:
df2021_8.iloc[8] 

Code                        AL832
Title       Mental Health Nursing
PointsR1                     440*
PointsR2                      431
Name: 8, dtype: object

In [44]:
# Re-casting as floats for analysis
#df2021_8['PointsR1'] = df2021_8['PointsR1'].astype(float)
#df2021_8['PointsR2'] = df2021_8['PointsR2'].astype(float)
#df2021_8

In [45]:
df2021_8.iloc[6] 

Code                                     AL820
Title       Mechanical and Polymer Engineering
PointsR1                                   327
PointsR2                                   NaN
Name: 6, dtype: object

In [46]:
df2021_8.loc[14] 

Code                          AL840
Title       Pharmaceutical Sciences
PointsR1                        346
PointsR2                        NaN
Name: 14, dtype: object

In [47]:
df2021_8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 949 entries, 0 to 948
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Code      949 non-null    object
 1   Title     949 non-null    object
 2   PointsR1  928 non-null    object
 3   PointsR2  258 non-null    object
dtypes: object(4)
memory usage: 29.8+ KB


In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#df2021 = pd.DataFrame(df2021,columns=['Course','RND 1'])
#df2021.plot(x ='Course', y='RND 1', kind = 'bar')
# Lets consider splitting this into college. Then compare points ranges. The compare college locations. 
# Find out how to select subsets as well.
# Points are for plotting and showing panadas functionality


#sns.displot(df2021['Course'], kde= False, bins = 15, color = 'green', edgecolor = 'orange', alpha=0.6) # setting kde as False in order to remove the density line as it is not relavant to task
## also altered alpha value to change transparency of the graph (adapted from Reference 13)
#plt.xlabel('Course')
#plt.ylabel('RND 1')
#plt.grid() # modifying grid lines so that they dont 'overpower' the data and used hex value for colour


In [49]:
# Top 5 Courses
#top_5 = df2021_8.sort_values(by="PointsR1", ascending=True).head()
#top_5

In [50]:
#ax = sns.countplot(x="Column", data=ds)
#sns.scatterplot(data=top_5, x='Title', y='PointsR1')
#sct.set_xticklabels(sct.get_xticklabels(), rotation=40)
#plt.xticks(
   # rotation=45, 
    #horizontalalignment='right',
    #fontweight='light',
     
#)
#plt.show()

In [51]:
#sns.displot(df2021_8['PointsR1'], kde= False, bins = 15, color = 'green', edgecolor = 'orange', alpha=0.6)

In [52]:
#top_medians = df2021_8[df2021_8["PointsR1"] < 230].sort_values("PointsR1")

#top_medians.plot(x="Course", y=["PointsR1"], kind="scatter")
#top_medians.count()

<br>

###### Concat and Join

In [53]:
courses2021_8 = df2021_8[['Code', 'Title']]
courses2021_8

,Code,Title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
944,WD211,Creative Computing
945,WD212,Recreation and Sport Management
946,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [54]:
courses2020 = df2020[['COURSE CODE2','COURSE TITLE']]
courses2020.columns = ['Code','Title']
courses2020
courses2020.sort_values('Code')

,Code,Title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,"First Year Art & Design (Common Entry,portfolio)"
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
1459,WD208,Manufacturing Engineering
1460,WD210,Software Systems Development
1461,WD211,Creative Computing
1462,WD212,Recreation and Sport Management


In [55]:
courses2019_8 = df2019[['Course Code','Course']]
courses2019_8.columns = ['Code','Title']
courses2019_8
courses2019_8.sort_values('Code')

,Code,Title
168,AC120,International Business
169,AC137,Liberal Arts
414,AD101,First Year Art & Design (Common Entry)
415,AD102,Graphic Design and Moving Image Design
416,AD103,Textile & Surface Design and Jewellery & Objects
...,...,...
925,WD200,Arts (options)
926,WD210,Software Systems Development
927,WD211,Creative Computing
928,WD212,Recreation and Sport Management


In [56]:
allcourses_L8 = pd.concat([courses2021_8, courses2020, courses2019_8], ignore_index=True)
allcourses_L8

,Code,Title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
3338,WD200,Arts (options)
3339,WD210,Software Systems Development
3340,WD211,Creative Computing
3341,WD212,Recreation and Sport Management


In [57]:
allcourses_L8.sort_values('Code')

,Code,Title
175,AC120,International Business
949,AC120,International Business
2581,AC120,International Business
950,AC137,Liberal Arts
2582,AC137,Liberal Arts
...,...,...
2412,WD230,Mechanical and Manufacturing Engineering
946,WD230,Mechanical and Manufacturing Engineering
3342,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [58]:
# Finds all extra copies of duplicated rows
allcourses_L8[allcourses_L8.duplicated()]

,Code,Title
949,AC120,International Business
950,AC137,Liberal Arts
952,AD102,Graphic Design and Moving Image Design (portfo...
955,AD204,Fine Art (portfolio)
956,AD211,Fashion Design (portfolio)
...,...,...
3338,WD200,Arts (options)
3339,WD210,Software Systems Development
3340,WD211,Creative Computing
3341,WD212,Recreation and Sport Management


In [59]:
# Returns a copy of the data frame with duplicates removed.
allcourses_L8.drop_duplicates()

,Code,Title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
3281,TL801,Animation Visual Effects and Motion Design
3282,TL802,"TV, Radio and New Media"
3283,TL803,Music Technology
3286,TL812,Computing with Digital Media


In [60]:
# Finds all extra copies of duplicated rows
allcourses_L8[allcourses_L8.duplicated(subset =['Code'])]

,Code,Title
949,AC120,International Business
950,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
952,AD102,Graphic Design and Moving Image Design (portfo...
953,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
3338,WD200,Arts (options)
3339,WD210,Software Systems Development
3340,WD211,Creative Computing
3341,WD212,Recreation and Sport Management


In [61]:
# Returns a copy of the data frame with duplicates removed = based only on code
allcourses_L8.drop_duplicates(subset = ['Code'], inplace = True, ignore_index = True)

In [62]:
allcourses_L8

,Code,Title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
1646,SG441,Environmental Science
1647,SG446,Applied Archaeology
1648,TL803,Music Technology
1649,TL812,Computing with Digital Media


In [63]:
allcourses_L8.sort_values('Code')

,Code,Title
175,AC120,International Business
176,AC137,Liberal Arts
311,AD101,First Year Art and Design (Common Entry portfo...
312,AD102,Graphic Design and Moving Image Design (portfo...
313,AD103,Textile and Surface Design and Jewellery and O...
...,...,...
944,WD211,Creative Computing
945,WD212,Recreation and Sport Management
946,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [64]:
allcourses_L8

,Code,Title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
1646,SG441,Environmental Science
1647,SG446,Applied Archaeology
1648,TL803,Music Technology
1649,TL812,Computing with Digital Media


##### Joins

<br>

###### Join to the points.
***

In [65]:
df2021_8

,Code,Title,PointsR1,PointsR2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


In [66]:
# Set the index to the code column
df2021_8.set_index('Code', inplace = True)
df2021_8.columns = ['Title','Points_R1_2021', 'Points_R2_2021']
df2021_8

,Title,Points_R1_2021,Points_R2_2021
Code,,,
AL801,Software Design for Virtual Reality and Gaming,300,NaN
AL802,Software Design in Artificial Intelligence for...,313,NaN
AL803,Software Design for Mobile Apps and Connected ...,350,NaN
AL805,Computer Engineering for Network Infrastructure,321,NaN
AL810,Quantity Surveying,328,NaN
...,...,...,...
WD211,Creative Computing,270,NaN
WD212,Recreation and Sport Management,262,NaN
WD230,Mechanical and Manufacturing Engineering,230,230


In [67]:
# Set the index to the code column
allcourses_L8.set_index('Code', inplace = True)


In [68]:
# Set the index to the code column
allcourses_L8 = allcourses_L8.join(df2021_8[[ 'Points_R1_2021']])
allcourses_L8

,Title,Points_R1_2021
Code,,
AL801,Software Design for Virtual Reality and Gaming,300
AL802,Software Design in Artificial Intelligence for...,313
AL803,Software Design for Mobile Apps and Connected ...,350
AL805,Computer Engineering for Network Infrastructure,321
AL810,Quantity Surveying,328
...,...,...
SG441,Environmental Science,NaN
SG446,Applied Archaeology,NaN
TL803,Music Technology,NaN


In [69]:
#df2021_8.loc['WD148']

In [70]:
df2020.columns

Index(['CATEGORY (i.e.ISCED description)', 'COURSE TITLE', 'COURSE CODE2',
       'R1 POINTS', 'R1 Random *', 'R2 POINTS', 'R2 Random*', 'EOS',
       'EOS Random *', 'EOS Mid-point', 'LEVEL', 'HEI', 'Test/Interview #',
       'avp', 'v', 'Column1', 'Column2', 'Column3', 'Column4', 'Column5',
       'Column6', 'Column7', 'Column8'],
      dtype='object')

In [71]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
df2020_r1 = df2020[['COURSE CODE2', 'R1 POINTS']]
df2020_r1.columns=['Code', 'Points_R1_2020']
df2020_r1

,Code,Points_R1_2020
0,AC120,209
1,AC137,252
2,AD101,#+matric
3,AD102,#+matric
4,AD103,#+matric
...,...,...
1459,WD208,188
1460,WD210,279
1461,WD211,271
1462,WD212,270


In [73]:
# Set the index to the code column
df2020_r1.set_index('Code', inplace = True)
df2020_r1

,Points_R1_2020
Code,
AC120,209
AC137,252
AD101,#+matric
AD102,#+matric
AD103,#+matric
...,...
WD208,188
WD210,279
WD211,271


In [74]:
# Join 2020 points to allcourses
allcourses_L8 = allcourses_L8.join(df2020_r1)
allcourses_L8

,Title,Points_R1_2021,Points_R1_2020
Code,,,
AL801,Software Design for Virtual Reality and Gaming,300,303
AL802,Software Design in Artificial Intelligence for...,313,332
AL803,Software Design for Mobile Apps and Connected ...,350,337
AL805,Computer Engineering for Network Infrastructure,321,333
AL810,Quantity Surveying,328,319
...,...,...,...
SG441,Environmental Science,NaN,NaN
SG446,Applied Archaeology,NaN,NaN
TL803,Music Technology,NaN,NaN


#### 2019 Join

In [75]:
df2019.columns

Index(['Course Code', 'Course', 'EOS', 'Mid'], dtype='object')

In [76]:
df2019_r1 = df2019[['Course Code', 'EOS']]
df2019_r1.columns = ['Code', 'Points_R1_2019']
df2019_r1

,Code,Points_R1_2019
0,AL801,304
1,AL802,301
2,AL803,309
3,AL805,329
4,AL810,307
...,...,...
925,WD200,221
926,WD210,271
927,WD211,275
928,WD212,274


In [77]:
# Set the index to the code column
df2019_r1.set_index('Code', inplace = True)
df2019_r1

,Points_R1_2019
Code,
AL801,304
AL802,301
AL803,309
AL805,329
AL810,307
...,...
WD200,221
WD210,271
WD211,275


In [78]:
# Join 2019 points to allcourses.
allcourses_L8 = allcourses_L8.join(df2019_r1)
allcourses_L8

,Title,Points_R1_2021,Points_R1_2020,Points_R1_2019
Code,,,,
AL801,Software Design for Virtual Reality and Gaming,300,303,304
AL802,Software Design in Artificial Intelligence for...,313,332,301
AL803,Software Design for Mobile Apps and Connected ...,350,337,309
AL805,Computer Engineering for Network Infrastructure,321,333,329
AL810,Quantity Surveying,328,319,307
...,...,...,...,...
SG441,Environmental Science,NaN,NaN,297
SG446,Applied Archaeology,NaN,NaN,289
TL803,Music Technology,NaN,NaN,264


In [79]:
# Ideas

Most points , least points
Biggest change year on year (between all)
Which college offers most course
How many courses offer placements in title
how many courses had Round 2
which courses accepted anyone
Level 7 no points for some courses
how many courses in points range x-x, 
What colleges had most points for engineering in title, science in title
did any courses cease operating YOY